#### Install

In [ ]:
```pip install ipywidgets```
```jupyter nbextension enable --py widgetsnbextension```

Can be needed for Mac:
Install Node.js for Mac. Download it from the website: https://nodejs.org/en/

```jupyter labextension install @jupyter-widgets/jupyterlab-manager```

In [1]:
# imports
import ipywidgets as w
import numpy as np
import pandas as pd
import csv
from collections import defaultdict
pd.set_option('display.max_colwidth', 0)

#### Explanation
This notebook is an annotator tool used for Dialogue Summarization Evaluation.\
The tool itself will explain how each element works. 

The parameters in the cell below will need to be adapted when stopping the annotation before finishing.\
In that case the load parameter needs to be set to 'True'and the index parameter to the value you were \
given when stopping the annotation.

To run the tool, run all cells below and the tool will be shown within the notbook.

#### Set parameters!

In [2]:
load = False            # if not the first time, and you want to load earlier saved data
index = 'explanation'   # if loading annotated data, enter index where you left off last time
annotator_number = 1        # note which annotator number you have been given
# update for your own use
path = 'data/original_human_judgment.jsonl'

#### Functionality

In [3]:
# setting up data
if load == False:
    data = pd.read_json(path, lines=True)

    def htmlivize(val):
        return val.replace('|','<br><br>')
    data.dialogue = data.dialogue.transform(htmlivize)

    line = f" <br> -------------------------------------------------------------------------------------------------- <br>"
    summary_start = "<font color='blue'> Summary: </font> "
    data['text'] = data['dialogue'] + line + summary_start + data['summary']

    df = pd.DataFrame(list(data.text), columns = ['texts'])

    
    df['coherence_results'] = 0
    df['consistency_results'] = 0
    df['relevance_results'] = 0
    df['fluency_results'] = 0
    df['comments'] = 'Any Comments?'

else:
    df = pd.read_csv(f'saved_df_ann{annotator_number}.csv', delimiter=';')


In [4]:
# explanation text
explanation =  """Welcome to the Annotation Tool.\
<br> You will be presented with 4 sliders that you can adjust. Once you have set the values, click 'Save and Next' and the slider values will be stored in the dataframe.\
<br> The sliders will be set to the stored values whenever you move to a different summary. You can always go back to a previous summary to make any changes.
<br> Use the 'Previous' and 'Next' buttons to move between summaries without storing any changes you made to the sliders. \
<br> When you have finished annotating all the summaries, press the 'Finish' button. The dataframe is stored in the data folder.\
<br> If you need to take a break, simply press the 'Pause' button. Your progress will be saved and you can resume where you left off at a later time.\
<br> When you you take a break or finish, youu can go ahead and close the kernel (and notebook).\
<br> <br> To get started, click 'Next' and simply go through each dialogue and annotate the summary."""

In [5]:
# sliders
coherence = w.IntSlider(value=3,
    min=1,
    max=5,
    step=1,
    description='Coherence:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d')

consistency = w.IntSlider(value=3,
    min=1,
    max=5,
    step=1,
    description='Consistency:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d')

relevance = w.IntSlider(value=3,
    min=1,
    max=5,
    step=1,
    description='Relevance:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d')

fluency = w.IntSlider(value=3,
    min=1,
    max=5,
    step=1,
    description='Fluency:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d')

comments = w.Textarea(value='Any comments?')

In [6]:
# Prepare button functionality
def next(b):
    global df, index, out, progression
    
    if index == 'explanation':
        index = 0
        progress_text.value=f'Current index: {index}'
        
        comments.value=df.loc[index,'comments']
        coherence.value = df.loc[index,'coherence_results']
        consistency.value = df.loc[index,'consistency_results']
        relevance.value = df.loc[index,'relevance_results']
        fluency.value = df.loc[index,'fluency_results']
        text = w.HTMLMath(value=df.loc[index, 'texts'])
        
        with out:
            out.clear_output()
            display(text)

    elif index == len(df):
        progress_text.value=f'Current index: {index}'
        text = w.HTMLMath(value='STOP <br> You are done!!!')
        with out:
            out.clear_output()
            display(text)

    else:
        progression.value = index+1
        
        # add current slider values to df
        df.loc[index,'coherence_results'] = coherence.value
        df.loc[index,'consistency_results'] = consistency.value
        df.loc[index,'relevance_results'] = relevance.value
        df.loc[index,'fluency_results'] = fluency.value
        df.loc[index, 'comments'] = comments.value

        index+=1
        if index == len(df):
            text = w.HTMLMath(value='STOP <br> You are done!!!')
            progress_text.value=f'Current index: {index}'
            
            with out:
                out.clear_output()
                display(text)
        
        else:
            comments.value=df.loc[index,'comments']
            coherence.value = df.loc[index,'coherence_results']
            consistency.value = df.loc[index,'consistency_results']
            relevance.value = df.loc[index,'relevance_results']
            fluency.value = df.loc[index,'fluency_results']

            progress_text.value=f'Current index: {index}'
            text = w.HTMLMath(value=df.loc[index, 'texts'])
            # was trying to update the texts show, but does not work
            with out:
                out.clear_output()
                display(text)

def small_next(b):
    # update position
    global df, index, out, progression

    if index == 'explanation':
        index = 0
        progression.value = index+1

        comments.value=df.loc[index,'comments']
        coherence.value = df.loc[index,'coherence_results']
        consistency.value = df.loc[index,'consistency_results']
        relevance.value = df.loc[index,'relevance_results']
        fluency.value = df.loc[index,'fluency_results']

    else:
        index += 1
    
    if index == len(df):
        index=0

    progress_text.value=f'Current index: {index}'
    
    comments.value=df.loc[index,'comments']
    coherence.value = df.loc[index,'coherence_results']
    consistency.value = df.loc[index,'consistency_results']
    relevance.value = df.loc[index,'relevance_results']
    fluency.value = df.loc[index,'fluency_results']


    # refresh display
    text = w.HTMLMath(value=df.loc[index, 'texts'])
    with out:
        out.clear_output()
        display(text)

def back(b):
    # update position
    global df, index, out, progression

    if index == 'explanation':
        progression.value = 0
        index = 'explanation'
        progress_text.value=f'Current index: {index}'
        text = w.HTMLMath(value=explanation)

    else:
        index -= 1
        if index == -1:
            progression.value = 0
            index = 'explanation'
            progress_text.value=f'Current index: {index}'
            text = w.HTMLMath(value=explanation)

        

        else:
            progression.value = index+1
            progress_text.value=f'Current index: {index}'

            comments.value = df.loc[index,'comments']
            coherence.value = df.loc[index,'coherence_results']
            consistency.value = df.loc[index,'consistency_results']
            relevance.value = df.loc[index,'relevance_results']
            fluency.value = df.loc[index,'fluency_results']

            text = w.HTMLMath(value=df.loc[index, 'texts'])
        
        with out:
            out.clear_output()
            display(text)

def pause_annotation(b):
    global index, df
    # store temp data
    df.to_csv(f'saved_df_ann{annotator_number}.csv', sep=';',index=False)

    text = w.HTMLMath(value=f'You paused annotation! Data is stored on your local drive. <br>\
                            Next time, add below index in top of notebook to continue where you left off.\
                            <br><br> Stopped at index: {index}')
    with out:
        out.clear_output()
        display(text)


def finish(b):
    global df
    # store data
    df.to_csv(f'saved_df_ann{annotator_number}.csv', sep=';',index=False)


    # notify user
    text = w.HTMLMath(value=f'You have finished annotating! The data is stored on the drive')
    with out:
        out.clear_output()
        display(text)


In [7]:
# Compose widget

# set initial Dialogue
if index == 'explanation':
    text = w.HTMLMath(value=explanation)
    
else:
    text = w.HTMLMath(value=df.loc[index, 'texts'])

out = w.Output()
with out:
    out.clear_output()
    display(text)

# save work button
forward = w.Button(description='Save and Next', icon='forward')
forward.on_click(next)
small_forward = w.Button(description='Next', icon='arrow-right')
small_forward.on_click(small_next)

previous =  w.Button(description='Previous',icon='arrow-left')
previous.on_click(back)

pause = w.Button(description='Pause and store data',icon='download')
pause.on_click(pause_annotation)
fin = w.Button(description='Finish',icon='check')
fin.on_click(finish)

progression_bar = w.IntProgress(
    value=0,
    min=0,
    max=len(df),
    description='Progress',
    bar_style='', # 'success', 'info', 'warning', 'danger' or ''
    style={'bar_color': 'cyan'},
    orientation='horizontal'
)

progress_text = w.HTMLMath(value=f'Current index: {index}')


# sliders & buttons
sliders = w.VBox([relevance, consistency, fluency, coherence])
buttons = w.HBox([previous, small_forward, forward, pause, fin])
progression = w.HBox([progression_bar, progress_text])

#### Run!

In [8]:
# run widget
display(out)
display(sliders)
display(comments)
display(progression)
display(buttons)

Output(outputs=({'output_type': 'display_data', 'data': {'text/plain': 'HTMLMath(value="Welcome to the Annotat…

Textarea(value='Any comments?')

In [10]:
# you can use this cell to check up on the current dataframe.
# But it is not needed, use the widget for storing data!
# df.iloc[index-2:index+1]